## Computer Vision Projects : skin detection 
#### Melika Heydari Rad 

___

In [1]:
import cv2
import numpy as np

In [3]:
print(cv2.__version__)
print(np.__version__)

4.8.0
1.23.4


___

In [7]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('Cris.jpeg')

bgr = img.copy()
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

In [9]:
hsv.shape

(650, 1155, 3)